In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from nltk.tokenize import RegexpTokenizer
import string
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

ImportError: cannot import name 'LinearSVM' from 'sklearn.svm' (/usr/local/lib/python3.10/dist-packages/sklearn/svm/__init__.py)

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
tokenizer = RegexpTokenizer(r'\w+')

def preprocess(text):

  text = text.replace('{html}', "") # Remove weblinks
  text = text.lower()
  text = REPLACE_BY_SPACE_RE.sub(' ', text)
  text = BAD_SYMBOLS_RE.sub('', text)
  text = ' '.join(word for word in text.split() if word not in stop_words)
  tokens = tokenizer.tokenize(text)
  cleanedText = " ".join(tokens)

  return cleanedText

In [ ]:
df = pd.read_csv('complaints.csv')
df.head()

In [ ]:
df.dropna(subset = ['Consumer complaint narrative'], inplace = True)
df['input'] = df['Consumer complaint narrative'].map(lambda x: preprocess(x))

##Understanding Word2Vec:
###https://www.geeksforgeeks.org/python-word-embedding-using-word2vec/

In [ ]:
X_train, y_train, X_test, y_yest = train_test_split(df['input'], df['Product'], test_size = 0.2, random_state = 42)

sentences =  [sentence.split() for sentence in X_train]
w2v_model = Word2Vec(sentences,
                     size=500,
                     window=5,
                     min_count=5
                     )

SyntaxError: invalid syntax (<ipython-input-6-a8c806e94c07>, line 3)

In [ ]:
def vectorize(sentence):

  words = sentence.split()
  words_vecs = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
  if len(words_vec) == 0:

    return np.zeros(100)

  words_vecs = np.array(words_vecs)

  return words_vecs.mean(axis=0)

In [ ]:
X_train = np.array([vectorize(sentence) for sentence in X_train])
X_test = np.array([vectorize(sentence) for sentence in X_test])

In [ ]:
logReg = LogisticRegression()
logReg.fit(X_train, y_train)
y_pred = logReg.predict(X_test)
print('Logistic Regression Accuracy:', accuracy_score(y_test, y_pred))

RF = RandomForestClassifier(n_estimators=200, max_depth=3, random_state=42)
RF.fit(X_train, y_train)
y_pred = RF.predict(X_test)
print('Random Forest Accuracy:', accuracy_score(y_test, y_pred))

LSVC = LinearSVC()
LSVC.fit(X_train, y_train)
y_pred = LSVC.predict(X_test)
print('SVM Accuracy:', accuracy_score(y_test, y_pred))